In [1]:
import pandas as pd

df = pd.read_csv('Reviews.csv')  # change filename as needed
df.head()


,User_id,Product_id,Rating,Date,Review,Label
0,923,0,3,12/8/2014,The food at snack is a selection of popular Gr...,-1
1,924,0,3,5/16/2013,This little place in Soho is wonderful. I had ...,-1
2,925,0,4,7/1/2013,ordered lunch for 15 from Snack last Friday. Â...,-1
3,926,0,4,7/28/2011,This is a beautiful quaint little restaurant o...,-1
4,927,0,4,11/1/2010,Snack is great place for a Â casual sit down l...,-1


In [2]:
# You may need to install nltk once
# !pip install nltk

import pandas as pd
import re
import nltk

# Download stopwords the first time you use NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SWVL-1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation and symbols
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)


In [4]:
# Load your data
df = pd.read_csv('Reviews.csv')

# Clean the reviews
df['cleaned_review'] = df['Review'].apply(clean_text)

# See the results
print(df[['Review', 'cleaned_review']].head())


                                              Review  \
0  The food at snack is a selection of popular Gr...   
1  This little place in Soho is wonderful. I had ...   
2  ordered lunch for 15 from Snack last Friday. Â...   
3  This is a beautiful quaint little restaurant o...   
4  Snack is great place for a Â casual sit down l...   

                                      cleaned_review  
0  food snack selection popular greek dishes appe...  
1  little place soho wonderful lamb sandwich glas...  
2  ordered lunch snack last friday â time nothing...  
3  beautiful quaint little restaurant pretty stre...  
4  snack great place â casual sit lunch especiall...  


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['cleaned_review']).toarray()
y = df['Label']   # Use your label column as target


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


MemoryError: Unable to allocate 6.42 GiB for an array with shape (287241, 3000) and data type float64

In [9]:
df_small = df.sample(n=2000, random_state=1)   # or whatever number/sample you used
print(df_small['Label'].value_counts())



Label
 1    1803
-1     197
Name: count, dtype: int64


In [11]:
# Split into fake and genuine
fake = df_small[df_small['Label'] == 1]
genuine = df_small[df_small['Label'] == -1]

# Downsample fake to match the number of genuine reviews
fake_balanced = fake.sample(n=len(genuine), random_state=42)

# Concatenate to make a balanced dataset
df_balanced = pd.concat([fake_balanced, genuine]).sample(frac=1, random_state=42).reset_index(drop=True)

# Check new balance
print(df_balanced['Label'].value_counts())


Label
 1    197
-1    197
Name: count, dtype: int64


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF on balanced data
tfidf = TfidfVectorizer(max_features=1000)
X = tfidf.fit_transform(df_balanced['cleaned_review']).toarray()
y = df_balanced['Label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6075949367088608
              precision    recall  f1-score   support

          -1       0.60      0.65      0.63        40
           1       0.61      0.56      0.59        39

    accuracy                           0.61        79
   macro avg       0.61      0.61      0.61        79
weighted avg       0.61      0.61      0.61        79



In [14]:
import joblib
joblib.dump(model, 'fake_review_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [15]:
(df_balanced[df_balanced['Label']==1]['Review'].head())

0    Great food!!! Interesting selection of beers o...
3    I love their Happy Hour $6-$8 Margaritas! So y...
6    I love this place! I love that it's somewhat o...
8    Awesome tacos. If got these in LA I would not ...
9    Stopped here for dessert after having dinner n...
Name: Review, dtype: object

In [16]:
(df_balanced[df_balanced['Label']==-1]['Review'].head())

1    I was looking forward to this place after read...
2    A new restaurant in Greenpoint that is actuall...
4    I love burritos and Mexican food. Dos Toros' C...
5                     Good Food at a reasonable price.
7    Lamb Lollies are delicious! Needs a bigger men...
Name: Review, dtype: object

In [7]:
# Separate the fake and genuine reviews
fake = df_small[df_small['Label'] == 1]
genuine = df_small[df_small['Label'] == -1]

# Balance the two classes
min_count = min(len(fake), len(genuine))
fake_balanced = fake.sample(min_count, random_state=42)
genuine_balanced = genuine.sample(min_count, random_state=42)
df_balanced = pd.concat([fake_balanced, genuine_balanced])

# Shuffle the rows
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


NameError: name 'df_small' is not defined

In [8]:
print(df_small['Label'].value_counts())

NameError: name 'df_small' is not defined

In [ ]:
tfidf = TfidfVectorizer(max_features=1000)  # You can adjust this lower if needed
X = tfidf.fit_transform(df['cleaned_review']).toarray()


MemoryError: Unable to allocate 2.68 GiB for an array with shape (359052, 1000) and data type float64

In [ ]:
df_small = df.sample(n=2000, random_state=1)   # Take a sample of 2000 rows
X = tfidf.fit_transform(df_small['cleaned_review']).toarray()
y = df_small['Label']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9075
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        37
           1       0.91      1.00      0.95       363

    accuracy                           0.91       400
   macro avg       0.45      0.50      0.48       400
weighted avg       0.82      0.91      0.86       400



C:\Users\SWVL-1\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SWVL-1\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SWVL-1\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [ ]:
import joblib
joblib.dump(model, 'fake_review_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
print(df_small['Label'].value_counts())


NameError: name 'df_small' is not defined

In [ ]:
df_small = df.sample(n=2000, random_state=1)   # or whatever number/sample you used


NameError: name 'df' is not defined

In [ ]:
def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation and symbols
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)
df['cleaned_review'] = df['Review'].apply(clean_text)

In [ ]:
import pandas as pd

df = pd.read_csv('your_reviews.csv')  # change filename as needed
df.head()



FileNotFoundError: [Errno 2] No such file or directory: 'your_reviews.csv'